In [ ]:
import os
from dotenv import load_dotenv
import array

from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings

import oracledb

In [ ]:
_ = load_dotenv()
un = os.getenv("ORACLE_USERNAME")
pw = os.getenv("ORACLE_PASSWORD")
dsn = os.getenv("ORACLE_DSN")
config_dir = "/tmp/wallet"
wallet_location = "/tmp/wallet"
wallet_password = os.getenv("WALLET_PASSWORD")

compartment_id = os.getenv("COMPARTMENT_ID")
service_endpoint = os.getenv("GENAI_ENDPOINT")

In [ ]:
embeddings = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
)

In [ ]:
with oracledb.connect(user=un, password=pw, dsn=dsn,
                      config_dir=config_dir, wallet_location=wallet_location,
                      wallet_password=wallet_password) as connection:
    with connection.cursor() as cursor:
        create_table_sql = """
            CREATE TABLE IF NOT EXISTS DEMO (
                ID NUMBER PRIMARY KEY,
                TEXT CLOB,
                VECTOR VECTOR(1024)
            )
        """
        cursor.execute(statement=create_table_sql)

In [ ]:
docs = [
    "人工知能は1956年に学問分野として設立されました。",
    "アラン・チューリングは人工知能の研究を大規模に行った最初の人物です。",
    "ロンドンのマイダ・ベールで生まれたチューリングは、イングランド南部で育ちました。",
]

In [ ]:
vectors = embeddings.embed_documents(texts=docs)
print(vectors)
data = [
    {"id": i, "text": docs[i], "vector": array.array("f", vectors[i])}
    for i in range(len(vectors))
]

print(data[0])
print("Vector Dims:", len(data[0]["vector"]))

In [ ]:
with oracledb.connect(user=un, password=pw, dsn=dsn, config_dir=config_dir, wallet_location=wallet_location, wallet_password=wallet_password) as connection:
    with connection.cursor() as cursor:
        insert_data_sql = """
            INSERT INTO DEMO (
                ID,
                TEXT,
                VECTOR
            ) VALUES (
                :id,
                :text,
                :vector
            )
        """
        cursor.executemany(statement=insert_data_sql, parameters=data)
    connection.commit()

In [ ]:
def output_type_handler(cursor, metadata):
    if metadata.type_code is oracledb.DB_TYPE_VECTOR:
        return cursor.var(metadata.type_code, arraysize=cursor.arraysize, outconverter=list)

In [ ]:
oracledb.defaults.fetch_lobs = False
with oracledb.connect(user=un, password=pw, dsn=dsn, config_dir=config_dir, wallet_location=wallet_location, wallet_password=wallet_password) as connection:
    connection.outputtypehandler = output_type_handler
    with connection.cursor() as cursor:
        cursor.execute(statement="SELECT * FROM DEMO")
        for row in cursor.fetchall():
            print("row:", row)

In [ ]:
query_vector = embeddings.embed_query("アラン・チューリングはだれですか？")
parameters = {"query_vector": array.array("f", query_vector)}
oracledb.defaults.fetch_lobs = False
with oracledb.connect(user=un, password=pw, dsn=dsn,
                      config_dir=config_dir, wallet_location=wallet_location,
                      wallet_password=wallet_password) as connection:
    connection.outputtypehandler = output_type_handler
    with connection.cursor() as cursor:
        vector_search_sql = """
            SELECT
                ID, TEXT
            FROM
                DEMO
            ORDER BY
                VECTOR_DISTANCE(:query_vector, VECTOR)
            FETCH FIRST 2 ROWS ONLY
        """
        cursor.execute(statement=vector_search_sql, parameters=parameters)
        for row in cursor.fetchall():
            print("row:", row)